In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.00001
IOU_THR = 0.7
PREDICTION_PATH = "dataset/predictions/06-2-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH).rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
test_df.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
107048,ID_GDhaPT.jpg,healthy,0.000007,1048.359375,645.468750,1080.000000,810.000000
139762,ID_i0dF6K.jpeg,healthy,0.000009,3961.124756,1397.812378,4031.999756,2075.062256
3093,ID_mcczYA.jpg,anthracnose,0.000031,615.000000,586.500000,1211.000000,1122.000000
73045,ID_LeHkk9.jpg,anthracnose,0.000005,343.750000,414.062500,606.250000,598.750000
220136,ID_ykD8wr.JPG,healthy,0.000014,0.896484,628.734375,361.382812,1571.437500


In [4]:
class_id = {j: i for i, j in enumerate(sorted(test_df["class"].unique()))}
test_df["class_id"] = test_df["class"].map(class_id)
class_id

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [5]:
test_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,healthy,0.673828,5.859375,0.976562,3970.702881,1664.062378,2
1,ID_cWEAQI.jpeg,healthy,0.108582,329.833954,1316.406128,1249.023315,1800.000000,2
2,ID_cWEAQI.jpeg,healthy,0.100891,177.734360,0.000000,1560.546753,496.093719,2
3,ID_cWEAQI.jpeg,anthracnose,0.052246,2710.937256,610.351501,3976.562256,1496.093628,0
4,ID_cWEAQI.jpeg,anthracnose,0.006123,3234.374756,864.257751,3828.124756,1316.406128,0


In [6]:
import torch

def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}


converted_labels = list(PREDICTION_IMAGE_PATH.glob("*"))
converted_labels = [i.name for i in converted_labels]

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	print(len(preds))
	print(list(preds.keys())[:10])
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [7]:
test_df.sample(2)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
236759,ID_N0xnjL.jpg,cssvd,0.000013,749.882812,104.611816,810.000000,260.771484,1
67130,ID_UbtyYv.jpg,healthy,0.000010,424.775391,0.000000,540.527344,21.357422,2


In [8]:
predictions = get_preds_data(test_df, thr=None)

len(predictions)

1626
['ID_A16nzu.jpg', 'ID_A1Euyz.jpg', 'ID_A1HcV0.jpeg', 'ID_A4ZdJC.jpeg', 'ID_A5SFUW.jpeg', 'ID_A6Fogi.jpeg', 'ID_ABDCyn.jpeg', 'ID_ACg6Qf.jpeg', 'ID_AFi8A1.jpg', 'ID_AHlc9P.jpg']


1626

In [9]:
# predictions["ID_CGnAYP.jpg"]

In [10]:
predictions["ID_CGnAYP.jpg"].keys()

dict_keys(['boxes', 'scores', 'labels'])

In [11]:
from torchvision.ops import nms

def apply_nms(predictions: dict[str, torch.Tensor], iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes.

    Args:
        predictions (dict): Dictionary containing keys ['boxes', 'scores', 'labels'].
        iou_threshold (float): Intersection over Union (IoU) threshold for NMS.

    Returns:
        dict: Filtered predictions after applying NMS.
    """
    filtered_predictions = {}
    for image_id, data in predictions.items():
        boxes = data["boxes"]
        scores = data["scores"]
        labels = data["labels"]

        if boxes.numel() == 0:
            # If no boxes, skip this image
            filtered_predictions[image_id] = data
            continue

        # Perform NMS for each class separately
        keep_indices = []
        for label in labels.unique():
            label_mask = labels == label
            label_boxes = boxes[label_mask]
            label_scores = scores[label_mask]
            indices = nms(label_boxes, label_scores, iou_threshold)
            keep_indices.extend(label_mask.nonzero(as_tuple=True)[0][indices].tolist())

        # Filter boxes, scores, and labels based on NMS results
        keep_indices = torch.tensor(keep_indices, dtype=torch.long)
        filtered_predictions[image_id] = {
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        }

    return filtered_predictions

# Apply NMS to predictions
filtered_predictions = apply_nms(predictions, iou_threshold=IOU_THR)

In [12]:
# for k, v in predictions.items():
#     r = filtered_predictions[k]
#     if len(v["boxes"]) != len(r["boxes"]):
#         print(k, len(v["boxes"]), len(r["boxes"]))
#         print("Boxes before NMS:", v["boxes"])
#         print("Boxes after NMS:", r["boxes"])
#         print("Scores before NMS:", v["scores"])
#         print("Scores after NMS:", r["scores"])
#         print("Labels before NMS:", v["labels"])
#         print("Labels after NMS:", r["labels"])
#         print()
#         break

In [13]:
filtered_data = []
id_to_label = {v: k for k, v in class_id.items()}
for image_id, data in filtered_predictions.items():
    boxes = data["boxes"].tolist()
    scores = data["scores"].tolist()
    labels = data["labels"].tolist()
    for box, score, label in zip(boxes, scores, labels):
        filtered_data.append({
            "Image_ID": image_id,
            "class": id_to_label[label],
            "confidence": score,
            "ymin": box[1],
            "xmin": box[0],
            "ymax": box[3],
            "xmax": box[2],
            "class_id": label
        })

filtered_df = pd.DataFrame(filtered_data)
filtered_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.052246,2710.937256,610.351501,3976.562256,1496.093628,0
1,ID_cWEAQI.jpeg,anthracnose,0.006123,3234.374756,864.257751,3828.124756,1316.406128,0
2,ID_cWEAQI.jpeg,anthracnose,0.005039,2130.859131,60.546871,3939.452881,1583.984253,0
3,ID_cWEAQI.jpeg,anthracnose,0.004280,1759.765503,0.000000,3380.859131,729.492126,0
4,ID_cWEAQI.jpeg,anthracnose,0.002632,2210.937256,0.000000,3035.156006,402.343719,0


In [14]:
len(filtered_df), len(test_df)

(243603, 243900)

In [15]:
filtered_df["class"].value_counts()

class
healthy        99249
cssvd          75240
anthracnose    69114
Name: count, dtype: int64

In [16]:
files = list(PREDICTION_IMAGE_PATH.glob("*"))

In [17]:
test_df: pd.DataFrame = filtered_df.copy()

In [18]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.05224609375,
  'ymin': 2710.937255859375,
  'xmin': 610.3515014648438,
  'ymax': 3976.562255859375,
  'xmax': 1496.0936279296875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.006122589111328125,
  'ymin': 3234.374755859375,
  'xmin': 864.2577514648438,
  'ymax': 3828.124755859375,
  'xmax': 1316.4061279296875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.005039215087890625,
  'ymin': 2130.859130859375,
  'xmin': 60.546871185302734,
  'ymax': 3939.452880859375,
  'xmax': 1583.9842529296875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.00428009033203125,
  'ymin': 1759.7655029296875,
  'xmin': 0.0,
  'ymax': 3380.859130859375,
  'xmax': 729.4921264648438,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.00263214111328125,
  

In [19]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [20]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)

In [21]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
223322,ID_yFxM4p.jpg,healthy,0.000047,723.960938,1606.500000,975.375000,1823.250000,2
102827,ID_m0or8l.jpg,anthracnose,0.000047,552.000000,570.000000,1250.000000,1122.000000,0
230744,ID_GX6BsX.jpeg,anthracnose,0.000009,185.062485,2179.406006,1840.781128,2860.593506,0
221615,ID_bwH4Zt.jpg,anthracnose,0.000017,970.500000,650.500000,1334.000000,1186.000000,0
89630,ID_g3WNqx.jpg,cssvd,0.000012,0.498047,2073.468750,74.756836,2448.000000,1
154593,ID_a76tQV.jpg,cssvd,0.000007,612.773438,0.000000,650.742188,2.900391,1
157404,ID_zfVZ2p.jpg,cssvd,0.000007,567.949219,0.000000,691.347656,0.000000,1
38448,ID_MZzLPS.jpg,cssvd,0.000005,554.238281,810.000000,600.644531,810.000000,1
54010,ID_Nbmn82.jpg,healthy,0.000152,0.000000,846.000000,58.937500,1278.000000,2
196645,ID_rV6NA8.jpg,cssvd,0.000004,678.164062,0.000000,1080.000000,0.000000,1


In [22]:
test_df["class"].value_counts()

class
healthy        99249
cssvd          75240
anthracnose    69114
Name: count, dtype: int64

In [23]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [24]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/06-2-submissions.csv'